<a href="https://colab.research.google.com/github/Jakson-Alves/IA-2021/blob/main/Classificador_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
# Vamos criar uma rede neural sequencial de multiplas camadas
from keras.models import Sequential 
# Conceito de fully conected, um reuronio conectado a todos outros
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

In [8]:
base = pd.read_csv('diabetes_data_upload_convertido.csv')
previsor = base.iloc[:, 2:16].values
classe = base.iloc[:, 16].values

labelencoder = LabelEncoder()
classe = labelencoder.fit_transform(classe)
classe_dummy = np_utils.to_categorical(classe)

In [ ]:

previsor_treinamento, previsor_test, classe_treinamento, classe_test = train_test_split(previsor, classe_dummy, test_size=0.25)

classificador = Sequential() # Instancia a classe / inicializa a rede neural
# units = quantidade de neuronios na primera camada oculta. Quantidade de variaveis + quantidade de classes / 2
# activation = Funcao de ativacao escolhida por vc
# input_dim = quantidade de variaveis no problema
classificador.add(Dense(units=8, activation='relu', input_dim=14))
classificador.add(Dropout(0.2))
classificador.add(Dense(units=8, activation='relu')) # Segunda camada oculta
classificador.add(Dropout(0.2))
# Saída da rede, ativado pela Softmax, pois temos multiplas classes
# Units = 3 pois temos 3 classes
classificador.add(Dense(units=2, activation='sigmoid')) 
# optimizer = Utiliza a funcao ADAM, para calcular o melhor valor na descida do gradiente
# loss = Calcula a perda entre as classes
# metrics = Define a métrica utilizada para calcular a assertividade
classificador.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# vamos mandar treinar agora!

classificador.fit(previsor_treinamento, classe_treinamento, batch_size = 10, epochs = 200)


In [ ]:
resultado = classificador.evaluate(previsor_test, classe_test)
print(resultado)

In [ ]:
# Salvar modelo para posterior inferência:
print('#'*10+' Salvando pesos do modelo... '+'#'*10)
classification_json = classificador.to_json()
# Aqui voce salva um json com a arquitetura da rede e metadados
with open('classify.json', 'w') as json_file:
    json_file.write(classification_json)
    print('Weight salvo...')
json_file.close()
classificador.save_weights('model.h5')

In [ ]:
from sklearn.metrics import confusion_matrix

# Matriz de confusão como métrica para avaliar a assertividade das classes
previsoes = classificador.predict(previsor_test)
print(previsoes)
previsoes = (previsoes > 0.5)


In [ ]:
# Carregando modelo para inferência
arquivo = open('classify.json', 'r')
estrutura_rede = arquivo.read()
arquivo.close()

base = pd.read_csv('teste_diabetes.csv')
previsor = base.iloc[:, 2:16].values
classe = base.iloc[:, 16].values

labelencoder = LabelEncoder()
classe = labelencoder.fit_transform(classe)
classe_dummy = np_utils.to_categorical(classe)

from keras.models import model_from_json
classificador = model_from_json(estrutura_rede)
# Carrega os pesos salvos no treinamento
classificador.load_weights('model.h5')

# Preve o resultado dos dados de teste do split
previsao = classificador.predict(previsor)
previsao = (previsao > 0.5)
#   Neg   1 0 
#   Posi  0 1 
print(previsor_test[0])
print(classe_test[0])

for element in previsao:
  if element[1]:
    print(element, " -> positivo")
  else:
   print(element, " -> Negativo")
    

